In [4]:
%load_ext autoreload
%autoreload 2
%pylab inline
import pandas as pd
import os

Populating the interactive namespace from numpy and matplotlib


In [5]:
import sys
sys.setrecursionlimit(10000) # 10000 is an example, try with different values

In [6]:
import sys
sys.path.append("/home/muncok/DL/projects/sv_system/")

In [7]:
import dnn.si_train as train
from dnn.parser import ConfigBuilder
from dnn.train import model as mod
from dnn.data import dataset as dset

# Dataframes
command: disjoint speaker split

In [30]:
dataframe_dir = '/home/muncok/DL/dataset/SV_sets/dataframes/'
data_dir = '/home/muncok/DL/dataset/SV_sets/speech_commands/'
command_df = pd.read_pickle(os.path.join(dataframe_dir,'Command_Dataframe.pkl'))

In [31]:
#TODO: correct splits
all_spks = list(command_df.spk.unique())
all_sents = command_df.sent.unique()

In [32]:
si_spks = list(command_df.spk.value_counts().index[10:])
sv_spks = list(command_df.spk.value_counts().index[:10])

In [33]:
command_df.sent.value_counts()

stop      2380
seven     2377
yes       2377
zero      2376
no        2375
up        2375
two       2373
four      2372
go        2372
one       2370
six       2369
on        2367
right     2367
nine      2364
down      2359
off       2357
five      2357
three     2356
left      2353
eight     2352
house     1750
marvin    1746
dog       1746
wow       1745
happy     1742
sheila    1734
cat       1733
tree      1733
bird      1731
bed       1713
Name: sent, dtype: int64

## Dataset Split

In [37]:
wanted_words = list(command_df.sent.value_counts().index[:10])

In [39]:
command_df[command_df.sent.isin(wanted_words)].spk.value_counts()

c1d39ce8    98
cb8f8307    75
c120e80e    74
cb2929ce    69
28ce0c58    60
e7ea8b76    57
d0faf7e4    54
7c1d8533    52
dbb40d24    51
bdee441c    51
9a7c1f83    51
324210dd    51
eb3f7d82    51
51055bda    51
b9f46737    51
742d6431    51
37dca74f    51
36050ef3    51
ec201020    51
692a88e6    51
e1469561    50
617de221    50
8f4c551f    50
953fe1ad    50
be7a5b2d    50
b83c1acf    50
35d1b6ee    50
f92e49f3    50
94de6a6a    50
cc6ee39b    50
            ..
936de313     1
3f6ba067     1
d1b4694a     1
3fd21070     1
4c62312c     1
928cc2ef     1
95458fae     1
d6265c56     1
8db999cf     1
9ba34294     1
4394fc7c     1
0a0b46ae     1
44fb4274     1
b1edd209     1
82e583e7     1
1538beb7     1
ecd4fd45     1
1e8044b4     1
d7d733d2     1
638685f2     1
5b34ad80     1
d8d12371     1
f57469fb     1
00176480     1
b1de039c     1
ea12e0d1     1
a6574a65     1
a07cfddf     1
0e4d22f1     1
6ef841e8     1
Name: spk, Length: 1845, dtype: int64

In [105]:
wanted_words

['stop', 'yes', 'seven', 'zero', 'up', 'no', 'two', 'go', 'four', 'one']

In [40]:
import csv
import random

# for word in wanted_words:
word = "right"
df = command_df[command_df.sent == word]

spk_counts = df.spk.value_counts()
valid_spks = list(spk_counts[spk_counts > 1].index)

import random
# random_idx = np.random.permutation(len(valid_spks))
train_spks = valid_spks[:450]
val_spks = valid_spks[450:]

right_train_df = df[df.spk.isin(train_spks)]
right_val_df = df[df.spk.isin(val_spks)]

tags = ['train', 'val', 'test']
sets = {'train':right_train_df, 'val':right_val_df, 'test':right_val_df}
manifest_dir = "./manifests/commands/"

data_dir = "/home/muncok/DL/dataset/SV_sets/speech_commands/vad"
for tag in tags:
    samples = []
    save_path = os.path.join(manifest_dir,'proto_{}_{}_manifest.csv'.format(word, tag))
    with open(save_path, 'w') as f:
        for index, row in sets[tag].iterrows():
            file_path = os.path.join(data_dir, row.sent, row.file)
            label = all_spks.index(row.spk)
            sample = ','.join([file_path, str(label)])
            samples.append(sample)
        random.shuffle(samples)
        writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
        writer.writerow(samples)
        print("{} was written".format(save_path))

si_df = df[df.spk.isin(si_spks)]
si_train = si_df.sample(frac=0.9)
si_val = si_df.drop(index = si_train.index)
si_test = si_val
print("[{}] train:{}, val:{}, test:{}".format(word, len(si_train), len(si_val), len(si_test)))

tags = ['train', 'val', 'test']
# choose a si split
sets = {'train':si_train, 'val':si_val, 'test':si_val}
manifest_dir = "./manifests/commands/"

data_dir = "/home/muncok/DL/dataset/SV_sets/speech_commands/vad"
for tag in tags:
    samples = []
    save_path = os.path.join(manifest_dir,'si_{}_{}_manifest.csv'.format(word, tag))
    with open(save_path, 'w') as f:
        for index, row in sets[tag].iterrows():
            file_path = os.path.join(data_dir, row.sent, row.file)
            label = all_spks.index(row.spk)
            sample = ','.join([file_path, str(label)])
            samples.append(sample)
        random.shuffle(samples)
        writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
        writer.writerow(samples)
        print("{} was written".format(save_path))

sv_df = df[df.spk.isin(sv_spks)]

tags = ['val']
# choose a si split
sets = {'val':sv_df}
manifest_dir = "./manifests/commands/"

data_dir = "/home/muncok/DL/dataset/SV_sets/speech_commands/vad"
for tag in tags:
    samples = []
    save_path = os.path.join(manifest_dir,'sv_{}_{}_manifest.csv'.format(word, tag))
    with open(save_path, 'w') as f:
        for index, row in sets[tag].iterrows():
            file_path = os.path.join(data_dir, row.sent, row.file)
            label = all_spks.index(row.spk)
            sample = ','.join([file_path, str(label)])
            samples.append(sample)
        random.shuffle(samples)
        writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
        writer.writerow(samples)
        print("{} was written".format(save_path))

# SI Train

In [10]:
model = "SimpleCNN"
dataset = "voxc"

global_config = dict(model=model, dataset=dataset,
                     no_cuda=False,  gpu_no=0,
                     n_epochs=100, batch_size=64,
                     lr=[0.01], schedule=[np.inf], dev_every=1, seed=0, use_nesterov=False,
                     cache_size=32768, momentum=0.9, weight_decay=0.00001,
                     num_workers=16, print_step=np.inf,
                     )

builder = ConfigBuilder(
                dset.SpeechDataset.default_config(),
                global_config)
parser = builder.build_argparse()
si_config = builder.config_from_argparse(parser)
train.set_seed(si_config)

### gated_cnn

In [11]:
seq_len         = 11
embd_size       = si_config['n_mels']
n_layers        = 10
kernel          = (5, embd_size)
out_chs         = 64
res_block_count = 5
ans_size = si_config['n_labels'] = 1881
si_model = mod.GatedCNN(seq_len, embd_size, n_layers, kernel, out_chs, res_block_count, ans_size)

In [7]:
si_config['n_labels'] = 1881
si_config['input_length'] = int(16000*1)
si_config['time_dim'] = int(16000*0.1)//160+1
si_config['train_manifest'] = "manifests/commands/si_train_manifest.csv"
si_config['val_manifest'] = "manifests/commands/si_val_manifest.csv"
si_config['test_manifest'] = "manifests/commands/si_test_manifest.csv"

In [8]:
si_model

GatedCNN(
  (conv_0): Conv2d(1, 64, kernel_size=(5, 40), stride=(1, 1), padding=(2, 0))
  (conv_gate_0): Conv2d(1, 64, kernel_size=(5, 40), stride=(1, 1), padding=(2, 0))
  (conv): ModuleList(
    (0): Conv2d(64, 64, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0))
    (1): Conv2d(64, 64, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0))
    (2): Conv2d(64, 64, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0))
    (3): Conv2d(64, 64, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0))
    (4): Conv2d(64, 64, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0))
    (5): Conv2d(64, 64, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0))
    (6): Conv2d(64, 64, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0))
    (7): Conv2d(64, 64, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0))
    (8): Conv2d(64, 64, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0))
    (9): Conv2d(64, 64, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0))
  )
  (conv_gate): ModuleList(
    (0): Conv2d(64, 64, ke

In [9]:
si_config['n_epochs'] = 50
si_config['input_format'] = 'mfcc'
si_config['output_file'] = ("models/commands/gated_cnn.pt")
train.train(si_config, model=si_model)

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

Process Process-6:
Traceback (most recent call last):
Process Process-9:
Process Process-14:
Process Process-11:
Process Process-8:
Traceback (most recent call last):
Process Process-13:
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Process Process-4:
Process Process-5:
Process Process-2:
Process Process-16:
Process Process-15:
Process Process-3:
Process Process-1:
Process Process-7:
Process Process-12:
Process Process-10:
Traceback (most recent call last):
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/muncok/anaconda3/envs/pytorch

  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()


Traceback (most recent call last):
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-49e6d159fe45>", line 4, in <module>
    train.train(si_config, model=si_model)
  File "/home/muncok/DL/projects/sv_system/dnn/si_train.py", line 201, in train
    loss.backward()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/autograd/variable.py", line 167, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph, retain_variables)
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/autograd/__init__.py", line 99, in backward
    variables, grad_variables, retain_graph)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/IP

  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/queues.py", line 341, in get
    with self._rlock:
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()


KeyboardInterrupt: 

  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/home/muncok/DL/projects/sv_system/dnn/data/manage_audio.py", line 17, in preprocess_audio
    data = librosa.feature.melspectrogram(data, sr=16000, n_mels=n_mels, hop_length=160, n_fft=480, fmin=20, fmax=4000)
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/queues.py", line 341, in get
    with self._rlock:
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/home/muncok/anaconda3/envs/pytorch/

### protonet model

In [12]:
si_config['model_class'] = mod.SimpleCNN
si_model = si_config['model_class'](small=True)

In [13]:
si_config['n_labels'] = 1881
si_config['input_length'] = int(16000*1)
si_config['time_dim'] = int(16000*0.1)//160+1
si_config['train_manifest'] = "manifests/commands/si_train_manifest.csv"
si_config['val_manifest'] = "manifests/commands/si_val_manifest.csv"
si_config['test_manifest'] = "manifests/commands/si_test_manifest.csv"

In [14]:
# si_model.load_partial("models/commands/no_aligned.pt")

In [15]:
from torch.autograd import Variable
import torch
import torch.nn as nn

time_dim = si_config['time_dim']
test_in = Variable(torch.zeros(1,1,time_dim,40), volatile=True)
test_out = si_model(test_in)
si_model.feat_size = test_out.size(1)
si_model.output = nn.Linear(test_out.size(1), si_config["n_labels"])

### random train

In [9]:
si_config['n_epochs'] = 50
si_config['input_format'] = 'mfcc'
si_config['output_file'] = ("models/commands/random.pt")
train.train(si_config, model=si_model)

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))


epoch #0, final dev accuracy: 0.02399057059961315
saving best model...


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))


epoch #1, final dev accuracy: 0.07052707930367505
saving best model...


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

...


epoch #47, final dev accuracy: 0.5021337040618956
saving best model...


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))


epoch #48, final dev accuracy: 0.48939192456479685


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))


epoch #49, final dev accuracy: 0.4931092843326886
final test accuracy: 0.4876692456479691


### batch mixed train

In [16]:
si_config['n_epochs'] = 50
si_config['input_format'] = 'mfcc'
si_config['output_file'] = ("models/commands/batch_random.pt")
train.train(si_config, model=si_model)

Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x7f7fe886f940>>


Traceback (most recent call last):
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 333, in __del__
Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x7f7fe886f940>>


Traceback (most recent call last):
    self._shutdown_workers()
Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x7f7fe886f940>>


  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 333, in __del__
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 319, in _shutdown_workers
Traceback (most recent call last):
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 333, in __del__
    self.data_queue.get()
    self._shutdown_workers()
Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x7f7fe886f940>>


  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 319, in _shutdown_workers
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/queues.py", line 344, in get
Traceback (most recent call last):
    self._shutdown_workers()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 319, in _shutdown_workers
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 333, in __del__
    self.data_queue.get()
    return _ForkingPickler.loads(res)
Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x7f7fe886f940>>
    self.data_queue.get()
    self._shutdown_workers()


...

    return _ForkingPickler.loads(res)
FileNotFoundError: [Errno 2] No such file or directory
    with _resource_sharer.get_connection(self._id) as conn:
    c = SocketClient(address)
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/resource_sharer.py", line 87, in get_connection
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/connection.py", line 614, in SocketClient
    fd = df.detach()
Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x7f7fe886f940>>
    c = Client(address, authkey=process.current_process().authkey)
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    s.connect(address)
Traceback (most recent call last):
  File "/home/muncok/anaconda3/envs/pytorch/l

Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x7f7fe886f940>>
Traceback (most recent call last):
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 333, in __del__
    self._shutdown_workers()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 319, in _shutdown_workers
    self.data_queue.get()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/queues.py", line 344, in get
    return _ForkingPickler.loads(res)
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/home/muncok/anaconda3/envs/py

Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x7f7fe886f940>>
Traceback (most recent call last):
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 333, in __del__
    self._shutdown_workers()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 319, in _shutdown_workers
    self.data_queue.get()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/queues.py", line 344, in get
    return _ForkingPickler.loads(res)
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/home/muncok/anaconda3/envs/py

Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x7f7fe886f940>>
Traceback (most recent call last):
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 333, in __del__
    self._shutdown_workers()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 319, in _shutdown_workers
    self.data_queue.get()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/queues.py", line 344, in get
    return _ForkingPickler.loads(res)
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/home/muncok/anaconda3/envs/py

AttributeError: Traceback (most recent call last):
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 55, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/muncok/DL/projects/sv_system/dnn/data/dataloader.py", line 71, in _batch_random_collate_fn
    np.shuffle(spliced_tensors)
AttributeError: module 'numpy' has no attribute 'shuffle'


### aligned train

In [ ]:
words = ['stop', 'yes', 'seven', 'zero', 'up', 'no', 'two', 'go', 'four', 'one']
train_loaders = []
dev_loaders = []
test_loaders = []
for word in words:
    for set_tag in ['train', 'val', 'test']:
        si_config['{}_manifest'.format(set_tag)] = os.path.join(manifest_dir, 'si_{}_{}_manifest.csv'.format(word, set_tag))
        # print(si_config['{}_manifest'.format(set_tag)])
    train_loader, dev_loader, test_loader = dloader.get_loader(si_config)
    train_loaders.append(train_loader); dev_loaders.append(dev_loader); test_loaders.append(test_loader)
loaders = [train_loaders, dev_loaders, test_loaders]

In [ ]:
### train
si_config['n_epochs'] = 200
si_config['output_file'] = ("models/commands/word_algined.pt")
train.aligned_train(si_config, model=si_model, loaders=loaders)

###  honk model

In [8]:
model = "cnn-strid-tpoo2"
dataset = "voxc"
si_config['model_class'] = mod.find_model(model)
train.set_seed(si_config)
si_model = si_config['model_class'](si_config)

NameError: name 'si_config' is not defined

### LSTM Model

In [ ]:
# lstm model
model = "lstm"
dataset = "command"

global_config = dict(model=model, dataset=dataset,
                     no_cuda=False,  gpu_no=0, 
                     n_epochs=30, batch_size=32,
                     lr=[0.01], schedule=[np.inf], dev_every=1, seed=0, use_nesterov=False,
                     cache_size=32768, momentum=0.9, weight_decay=0.00001, 
                     num_workers=8, print_step=100,
                     h_dim=500, n_layer=1)

builder = hk.ConfigBuilder(
                mod.find_config(model),
                mod.SpeechDataset.default_config(dataset),
                global_config)
parser = builder.build_argparse()
si_config = builder.config_from_argparse(parser)
si_config['model_class'] = mod.find_model(model)
hk.set_seed(si_config)

In [ ]:
si_config['n_labels'] = 2000
manifest_dir = "manifests/commands/"
for tag in ['train', 'val', 'test']:
    si_config['{}_manifest'.format(tag)] =  os.path.join(manifest_dir,'si_{}_{}_manifest.csv'.format("command", tag))

si_config['bn_size'] = 256
si_model = si_config['model_class'](si_config)
si_config['input_file'] = ""
si_config['output_file'] = "models/si_command_lstm_model.pt"

### SV Score

In [8]:
import pickle
import itertools
import torch
from dnn.sv_score import sv_score
from dnn.train.model import init_protonet, init_seed
from dnn.data.dataloader import init_sv_loaders
from dnn.parser import get_sv_parser

In [9]:
def secToSample(sec):
    return int(16000 * sec)

In [10]:
options = get_sv_parser().parse_args(args=["-cTr=5", "-cVa=5"])
options.n_dct_filters = 40
options.n_mels = 40
options.timeshift_ms = 100
options.data_folder = "/home/muncok/DL/dataset/SV_sets"
options.window_size= 0.025
options.window_stride= 0.010
options.cache_size = 32768
options.input_format = "mfcc"
init_seed(options)

In [11]:
options.val_manifest = "manifests/commands/sv_right_val_manifest.csv"

In [12]:
options.num_support_val = 1
options.num_query_val = 4
options.classes_per_it_val = 5
options.splice_length = secToSample(0.1)//160 +1
# options.input_length = secToSample(1.0)

In [13]:
from dnn.train.model import init_protonet
# lda = pickle.load(open("models/lda/lda_voxc_0.2.pkl", "rb"))

In [14]:
options.input = "models/commands/word_algined.pt"
aligned_model = init_protonet(options, fine_tune=False, small=True)

val_dataloader = init_sv_loaders(options)

sv_score(options, val_dataloader, aligned_model, ["full"])

FileNotFoundError: [Errno 2] No such file or directory: 'models/commands/word_algined.pt'

In [27]:
options.input = "models/commands/random.pt"
no_aligned_model = init_protonet(options, fine_tune=False, small=True)

val_dataloader = init_sv_loaders(options)
# lda = pickle.load(open("models/lda/lda_voxc_0.2.pkl", "rb"))
sv_score(options, val_dataloader, no_aligned_model, ["full"])

models/commands/random.pt is loaded










{'full': (0.1235, 0.016771260780874847)}

In [15]:
options.input = "../sv_system/models/voxc/si_train/full_train/si_voxc_0.1s_full.pt"
batch_random_model = init_protonet(options, fine_tune=False, small=True)
val_dataloader = init_sv_loaders(options)
# lda = pickle.load(open("models/lda/lda_voxc_0.2.pkl", "rb"))
sv_score(options, val_dataloader, batch_random_model, ["full"])

../sv_system/models/voxc/si_train/full_train/si_voxc_0.1s_full.pt is loaded


{'full': (0.2585, 0.011285818931093927)}